In [2]:
# !pip install attention

In [25]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model, Model

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau

from attention import Attention

In [26]:
data = np.load('train_test_x_y.npz')
train_x, test_x, train_y, test_y  = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'] 

train_y = train_y.reshape(-1,1)
test_y  = test_y.reshape(-1,1)

train_x.shape, test_x.shape, train_y.shape, test_y.shape

input_shape = train_x.shape[1:]

In [27]:
####
import tensorflow.keras.backend as K
def clear_sess():
  try:
    del model 
    del history 
  except:
    pass
  from tensorflow.keras import backend as K
  K.clear_session()
  import gc
  gc.collect()



  return None
clear_sess()

In [28]:
input_1 = Input(shape=input_shape)
lstm_1 = LSTM(26, return_sequences=True)(input_1)
att = Attention(32)(lstm_1)
output = Dense(1,activation="sigmoid")(att)

model = Model(input_1,output)

In [29]:
# model.summary()

In [30]:
def keras_mcc(y_true, y_pred):
    tp = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    tn = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    fp = K.sum(K.round(K.clip((1 - y_true) * y_pred, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true * (1 - y_pred), 0, 1)))

    num = tp * tn - fp * fn
    den = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    return num / K.sqrt(den + K.epsilon())

In [31]:
metrics_c = [keras.metrics.AUC(name="auc_roc",curve="ROC"),keras.metrics.AUC(name="auc_pr",curve="PR"),keras_mcc]


In [32]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005), loss='binary_crossentropy', metrics=metrics_c, weighted_metrics=[])


In [33]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 26)]          0         
                                                                 
 lstm (LSTM)                 (None, 48, 26)            5512      
                                                                 
 attention (Attention)       (None, 32)                2340      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 7,885
Trainable params: 7,885
Non-trainable params: 0
_________________________________________________________________


In [34]:
# callbacks = [keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True),]
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.99,patience=20, min_lr=0.005, verbose=1)

checkpoint_filepath_4 = 'weights-improvement-val-keras-mcc.hdf5'
model_checkpoint_callback_4 = ModelCheckpoint(filepath=checkpoint_filepath_4,save_weights_only=False,monitor='val_keras_mcc',mode='max',save_best_only=True)


In [35]:
train_history=model.fit(train_x,train_y,validation_split=0.2, epochs=300,batch_size=128,callbacks=[reduce_lr, model_checkpoint_callback_4])


Epoch 1/300


2023-03-03 20:54:39.490551: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce GTX 1650" frequency: 1515 num_cores: 14 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 1048576 shared_memory_size_per_multiprocessor: 65536 memory_size: 2748448768 bandwidth: 192032000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-03-03 20:54:47.231458: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


105/115 [==========================>...] - ETA: 0s - loss: 0.3033 - auc_roc: 0.7284 - auc_pr: 0.2437 - keras_mcc: 0.1973

2023-03-03 20:54:51.205868: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce GTX 1650" frequency: 1515 num_cores: 14 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 1048576 shared_memory_size_per_multiprocessor: 65536 memory_size: 2748448768 bandwidth: 192032000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


115/115 [==============================] - 14s 10ms/step - loss: 0.2999 - auc_roc: 0.7332 - auc_pr: 0.2477 - keras_mcc: 0.1950 - val_loss: 0.2797 - val_auc_roc: 0.7880 - val_auc_pr: 0.3419 - val_keras_mcc: 0.2317 - lr: 0.0050
Epoch 2/300
115/115 [==============================] - 1s 5ms/step - loss: 0.2586 - auc_roc: 0.8036 - auc_pr: 0.4017 - keras_mcc: 0.2765 - val_loss: 0.2742 - val_auc_roc: 0.7947 - val_auc_pr: 0.3583 - val_keras_mcc: 0.1901 - lr: 0.0050
Epoch 3/300
115/115 [==============================] - 1s 5ms/step - loss: 0.2491 - auc_roc: 0.8237 - auc_pr: 0.4360 - keras_mcc: 0.3064 - val_loss: 0.2800 - val_auc_roc: 0.7899 - val_auc_pr: 0.3376 - val_keras_mcc: 0.2449 - lr: 0.0050
Epoch 4/300
115/115 [==============================] - 1s 5ms/step - loss: 0.2399 - auc_roc: 0.8377 - auc_pr: 0.4777 - keras_mcc: 0.3736 - val_loss: 0.2810 - val_auc_roc: 0.7883 - val_auc_pr: 0.3347 - val_keras_mcc: 0.2237 - lr: 0.0050
Epoch 5/300
115/115 [==============================] - 1s 5ms/step